In [1]:
from typing import Any
import pyodbc
import pandas as pd
from configparser import ConfigParser

In [ ]:
class DataReader():
    db_name = 'One Practice Sample'
    server = 'CMDLHRDB01'
    database = 'One Practice Sample'
    trusted_connection = 'yes'  # Use Windows authentication
    driver = '{SQL Server}'

    def getData(self,table_name):
        connection_string = f"DRIVER={self.driver};SERVER={self.server};DATABASE={self.database};TRUSTED_CONNECTION={self.trusted_connection}"
        connection = pyodbc.connect(connection_string)
        data = pd.read_sql(f"select * from [One Practice Sample ].[dbo].[{table_name}]", connection)
        connection.close()
        return data
    
    def getFilteredData(self, table_name, columns):
        connection_string = f"DRIVER={self.driver};SERVER={self.server};DATABASE={self.database};TRUSTED_CONNECTION={self.trusted_connection}"
        connection = pyodbc.connect(connection_string)
        clns = "".join([f"[{col}]," for col in columns])
        clns = clns[:-1]
        data = pd.read_sql(f"select {clns} from [One Practice Sample ].[dbo].[{table_name}]", connection)
        connection.close()
        return data
    
    
class PreProcessor():
    og_column = "Patient ID"
    column_fixed = "PatientID"
    datatype = int
    def __init__(self, df):
        self.df = df
        

    
    def getRename(self, table_name):
        table_name.rename(columns = {self.og_column:self.column_fixed}, inplace=True, )
        return table_name
    
    def fixDatatype(self, table_name):
        table_name[self.column_fixed] = table_name[self.column_fixed].astype(self.datatype)
        return table_name
    

    

class InfoProducer(PreProcessor):
    def __init__(self, df):
        super().__init__(df=df)
        self.df = df
        self.df = super().getRename(table_name=self.df)
        self.df = super().fixDatatype(table_name=self.df)
    
    # def __init__(self, patient_ID:int, merged):
    #     self.patient_ID = patient_ID
    #     self.merged = merged|
        # merged[merged['PatientID']==patient_ID]

    def merger(self, obj):
        merged = obj.df.merge(self.df, on = 'PatientID', how = 'inner')       # merged = vitals.merge(notes, on = 'PatientID', how = 'inner')
        self.merged_obj = InfoProducer(merged)
        return self.merged_obj
    
    def getiddata(self, id):
        return self.df[self.df.PatientID==id]